In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [3]:
df0 = pd.read_pickle('.\\data\\MRegularSeasonDetailedResults.pkl')
df1 = df0.loc[df0['Season'] >= 2021].copy(deep=True)

In [4]:
kp_eff = pd.read_pickle('.\\data\\\kp_eff_stats.pkl')
teams = pd.read_pickle('.\\data\\MTeams.pkl')
spelling = pd.read_pickle('.\\data\\MTeamSpellings_lex.pkl')
kp_height = pd.read_pickle('.\\data\\kp_height.pkl')
kp_four_factors = pd.read_pickle('.\\data\\kp_four_factors.pkl')
kp_point_dist = pd.read_pickle('.\\data\\kp_point_dist.pkl')

In [5]:
kp_eff2 = kp_eff.merge(spelling, how='left', left_on = 'Team', right_on = 'TeamNameSpelling')
kp_height2 = kp_height.merge(spelling, how='left', left_on = 'Team', right_on = 'TeamNameSpelling')
kp_four_factors2 = kp_four_factors.merge(spelling, how='left', left_on = 'Team', right_on = 'TeamNameSpelling')
kp_point_dist2 = kp_point_dist.merge(spelling, how='left', left_on = 'Team', right_on = 'TeamNameSpelling')

In [6]:
#Efficiency Table 

eff_cols = ['TeamID', 'season', 'Tempo-Adj', 'Avg. Poss Length-Offense', 'Avg. Poss Length-Offense.Rank', 
            'Avg. Poss Length-Defense', 'Avg. Poss Length-Defense.Rank', 'Off. Efficiency-Adj', 
            'Off. Efficiency-Adj.Rank', 'Def. Efficiency-Adj', 'Def. Efficiency-Adj.Rank'
]
   
kp_eff3 = kp_eff2.loc[:, eff_cols]
kp_eff3.columns =  ['TeamID', 'season', 'tempo_adj', 'avg_poss_len_off', 'avg_poss_len_off_rnk', 
                    'avg_poss_len_def', 'avg_poss_len_def_rnk', 'off_eff_adj', 'off_eff_adj_rnk',
                    'def_eff_adj', 'def_eff_adj_rnk']
   

In [7]:
#Height Table 
hgt_cols = ['TeamID', 'season', 'AvgHgt', 'AvgHgt.Rank', 'EffHgt', 'EffHgt.Rank', 'C-Hgt', 
            'PF-Hgt', 'SF-Hgt', 'SG-Hgt', 'PG-Hgt', 'Experience', 'Bench', 'Continuity',
           ]
kp_height3 = kp_height2.loc[:, hgt_cols]
kp_height3.columns = ['TeamID', 'season', 'avg_hgt', 'avg_hgt_rank', 'eff_hgt', 'eff_hgt_rank', 'c_hgt', 
                                          'pf_hgt', 'sf_hgt', 'sg_hgt', 'pg_hgt', 'exp', 'bench', 'continuity',]

In [8]:
#Four Factos Table 
ff_cols = ['TeamID', 'season', 'Off-eFG%', 'Off-TO%', 'Off-OR%', 'Off-FTRate', 
           'Def-eFG%', 'Def-TO%', 'Def-OR%', 'Def-FTRate',
]
           
kp_four_factors3 = kp_four_factors2.loc[:, ff_cols]
kp_four_factors3.columns = ['TeamID', 'season', 'off_efg_pct', 'off_to_pct', 'off_or_pct', 'off_ft_pct', 
                                                'def_efg_pct', 'def_to_pct', 'def_or_pct', 'def_ft_pct',]

In [9]:
pd_cols = ['TeamID', 'season', 'Off-FT', 'Off-2P', 'Off-3P', 'Def-FT', 
           'Def-2P', 'Def-3P', ]
kp_point_dist3 = kp_point_dist2.loc[:, pd_cols]
kp_point_dist3.columns = ['TeamID', 'season', 'off_ft', 'off_2p', 'off_3p',
                                              'def_ft', 'def_2p', 'def_3p', ]

In [10]:
kp_master = kp_eff3.merge(kp_height3.merge(kp_four_factors3.merge(kp_point_dist3, how='left', on=['TeamID', 'season']), how='left', on=['TeamID', 'season']), how='left', on = ['TeamID', 'season'])

In [11]:
kp_master.head()

,TeamID,season,tempo_adj,avg_poss_len_off,avg_poss_len_off_rnk,avg_poss_len_def,avg_poss_len_def_rnk,off_eff_adj,off_eff_adj_rnk,def_eff_adj,...,def_efg_pct,def_to_pct,def_or_pct,def_ft_pct,off_ft,off_2p,off_3p,def_ft,def_2p,def_3p
0,1164.0,2021,76.8,14.0,1,16.1,13,91.0,334,102.6,...,47.9,20.4,32.5,27.0,21.9,41.9,36.1,17.1,50.3,32.6
1,1184.0,2021,74.8,15.4,16,16.1,17,101.0,197,101.5,...,51.0,24.3,30.2,32.2,15.0,49.4,35.6,18.8,53.2,28.0
2,1376.0,2021,74.6,15.2,12,16.7,64,103.9,139,98.9,...,53.2,22.0,31.3,39.0,16.3,55.7,28.0,21.8,48.0,30.1
3,1154.0,2021,74.2,15.5,18,16.7,62,104.0,136,109.1,...,52.8,13.9,25.7,25.9,18.7,46.6,34.7,16.0,57.1,26.9
4,1284.0,2021,74.2,14.6,4,17.8,271,99.0,237,105.1,...,48.6,20.9,28.5,38.4,21.8,47.9,30.4,21.5,46.4,32.1


In [12]:
df1['game_count'] = 1
df1['WLoc'] = np.where((df1['WLoc'] == 'N') & (df1['WTeamID'] > df1['LTeamID']), 'H', df1['WLoc'] )
df1['WLoc'] = np.where(df1['WLoc'] == 'N', 'A', df1['WLoc'])
df1.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,game_count
92832,2021,23,1101,70,1190,47,A,0,20,49,...,14,17,11,27,5,22,6,2,0,1
92833,2021,23,1104,81,1240,57,H,0,30,77,...,10,17,18,28,10,20,5,2,0,1
92834,2021,23,1111,81,1354,61,A,0,32,62,...,6,11,12,18,12,9,13,0,14,1
92835,2021,23,1113,94,1348,88,A,0,27,61,...,21,35,7,26,14,16,8,2,0,1
92836,2021,23,1114,71,1341,66,A,0,22,51,...,4,4,7,15,14,24,12,1,0,1


In [13]:
df1['home_id'] = np.where(df1['WLoc'] == 'H', df1['WTeamID'], df1['LTeamID'])
df1['away_id'] = np.where(df1['WLoc'] == 'H', df1['LTeamID'], df1['WTeamID'])
df1['home_score'] = np.where(df1['WLoc'] == 'H', df1['WScore'], df1['LScore'])
df1['away_score'] = np.where(df1['WLoc'] == 'H', df1['LScore'], df1['WScore'])
df1['home_win'] = np.where(df1['WLoc'] == 'H', 1, 0)

In [14]:
result_cols = ['Season', 'home_id', 'away_id', 'home_score', 'away_score', 'home_win']
df_results = df1.loc[:, result_cols]
df_results.columns = ['season', 'home_id', 'away_id', 'home_score', 'away_score', 'home_win']

In [15]:
kp_home = kp_master.copy(deep=True)
col_home = list(kp_home.columns)
col_home2 = [f'home_{i}' if _ > 1 else i for _, i in enumerate(col_home)]
kp_home.columns = col_home2

kp_away = kp_master.copy(deep=True)
col_away = list(kp_away.columns)
col_away2 = [f'away_{i}' if _ > 1 else i for _, i in enumerate(col_away)]
kp_away.columns = col_away2

df_results1 = df_results.merge(kp_home, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
df_results2 = df_results1.merge(kp_away, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])

In [16]:
df_results2.head()

,season,home_id,away_id,home_score,away_score,home_win,TeamID_x,home_tempo_adj,home_avg_poss_len_off,home_avg_poss_len_off_rnk,...,away_def_efg_pct,away_def_to_pct,away_def_or_pct,away_def_ft_pct,away_off_ft,away_off_2p,away_off_3p,away_def_ft,away_def_2p,away_def_3p
0,2021,1190,1101,47,70,0,1190.0,65.4,18.1,256,...,46.0,26.5,27.9,41.1,18.2,50.5,31.4,23.5,50.9,25.5
1,2021,1104,1240,81,57,1,1104.0,73.3,14.2,2,...,48.1,15.7,27.9,25.5,16.9,50.4,32.7,15.3,48.9,35.9
2,2021,1354,1111,61,81,0,1354.0,71.6,16.6,89,...,50.0,19.8,30.2,23.7,20.1,43.8,36.1,14.6,53.0,32.4
3,2021,1348,1113,88,94,0,1348.0,69.0,16.9,132,...,52.5,20.7,30.1,36.1,19.8,51.3,28.9,20.4,49.7,29.9
4,2021,1341,1114,66,71,0,1341.0,69.2,16.7,107,...,50.7,20.1,28.0,27.8,21.0,54.4,24.6,16.6,49.4,33.9


In [17]:
x_cols = list(df_results2.columns)[7:]

X = df_results2.loc[:, x_cols]
y = df_results2.loc[:, 'home_score']

In [18]:
X[x_cols] = X[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [19]:
xgb_cl = xgb.XGBClassifier()
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [21]:
regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [22]:
pd.DataFrame(regressor.feature_importances_.reshape(1, -1), columns=x_cols)

,home_tempo_adj,home_avg_poss_len_off,home_avg_poss_len_off_rnk,home_avg_poss_len_def,home_avg_poss_len_def_rnk,home_off_eff_adj,home_off_eff_adj_rnk,home_def_eff_adj,home_def_eff_adj_rnk,home_avg_hgt,...,away_def_efg_pct,away_def_to_pct,away_def_or_pct,away_def_ft_pct,away_off_ft,away_off_2p,away_off_3p,away_def_ft,away_def_2p,away_def_3p
0,0.023068,0.061678,0.062665,0.012349,0.008523,0.102557,0.073785,0.005831,0.006031,0.003911,...,0.016682,0.005362,0.014691,0.00407,0.005061,0.006216,0.005258,0.007162,0.005628,0.005218


In [23]:
y_pred = regressor.predict(X_test)

In [24]:
mean_squared_error(y_test, y_pred)

91.16252097614424

In [25]:
df_home_output = pd.DataFrame({'predicted_home' : y_pred, 
                               'actual_home' : y_test})

In [26]:
#y_away = df_results2.loc[:, 'home_score']

In [27]:
y_away_train = df_results2.loc[y_train.index, 'away_score']
y_away_test = df_results2.loc[y_test.index, 'away_score']

In [131]:
regressor_away = xgb.XGBRegressor(
    n_estimators=150,
    reg_lambda=2,
    gamma=2,
    max_depth=1
)

In [132]:
regressor_away.fit(X_train, y_away_train)
pd.DataFrame(regressor_away.feature_importances_.reshape(1, -1), columns=x_cols)

,home_tempo_adj,home_avg_poss_len_off,home_avg_poss_len_off_rnk,home_avg_poss_len_def,home_avg_poss_len_def_rnk,home_off_eff_adj,home_off_eff_adj_rnk,home_def_eff_adj,home_def_eff_adj_rnk,home_avg_hgt,...,away_def_efg_pct,away_def_to_pct,away_def_or_pct,away_def_ft_pct,away_off_ft,away_off_2p,away_off_3p,away_def_ft,away_def_2p,away_def_3p
0,0.083188,0.010444,0.00681,0.0,0.003061,0.005537,0.0,0.126829,0.103741,0.0,...,0.003645,0.00512,0.005592,0.007044,0.002653,0.004767,0.003335,0.0,0.00338,0.0


In [133]:
y_away_pred = regressor_away.predict(X_test)

In [134]:
mean_squared_error(y_away_test, y_away_pred)

85.53137916693396

In [135]:
df_away_output = pd.DataFrame({'predicted_away' : y_away_pred, 
                               'actual_away' : y_away_test})

In [136]:
df_test = df_away_output.join(df_home_output)

In [137]:
df_test['pred_home_win'] = np.where(df_test['predicted_home']>df_test['predicted_away'], 1, 0)
df_test['actual_home_win'] = np.where(df_test['actual_home']>df_test['actual_away'], 1, 0)

In [138]:
df_test

,predicted_away,actual_away,predicted_home,actual_home,pred_home_win,actual_home_win
8733,76.024933,71,81.376381,63,1,0
13042,74.207069,65,69.774567,66,0,1
3286,66.739929,60,70.233070,68,1,1
10695,59.061554,41,82.926453,72,1,1
15450,68.444771,75,69.933502,76,1,1
...,...,...,...,...,...,...
13968,65.188057,71,71.302574,77,1,1
2671,81.113228,67,73.242882,81,0,1
9687,67.823288,63,65.208244,61,0,0
2350,74.817833,76,81.996582,75,1,0


In [139]:
from sklearn.metrics import f1_score

f1_score(df_test['actual_home_win'], df_test['pred_home_win'], average='macro') 

0.7113457168197705

In [140]:
round1 = pd.read_pickle('.\\data\\round1_games.pkl').dropna()
round1['season'] = 2023

In [141]:
round1_0 = round1.merge(kp_home, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
round1_1 = round1_0.merge(kp_away, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
round1_2 = round1.merge(kp_home, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
round1_3 = round1_0.merge(kp_away, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])

In [142]:
round1_0 = round1.merge(kp_home, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
round1_1 = round1_0.merge(kp_away, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
round1_2 = round1.merge(kp_home, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
round1_3 = round1_0.merge(kp_away, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
rnd1_x = round1_1.loc[:, x_cols]

rnd1_x[x_cols] = rnd1_x[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

rnd1_x1 = round1_3.loc[:, x_cols]

rnd1_x1[x_cols] = rnd1_x1[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

rnd1_away = regressor_away.predict(rnd1_x)
rnd1_home = regressor.predict(rnd1_x)
rnd1_away2 = regressor_away.predict(rnd1_x1)
rnd1_home2 = regressor.predict(rnd1_x1)

In [143]:
rnd1_away = regressor_away.predict(rnd1_x)
rnd1_home = regressor.predict(rnd1_x)
rnd1_away2 = regressor_away.predict(rnd1_x1)
rnd1_home2 = regressor.predict(rnd1_x1)

In [144]:
rnd1_teams = round1_1[['home_id', 'away_id']]

In [145]:
rnd1_scores = pd.DataFrame({'away_score0' : rnd1_away,
                            'away_score1' : rnd1_away2,
                            'home_score0' : rnd1_home, 
                            'home_score1' : rnd1_home2})

rnd1_scores['home_score'] = (rnd1_scores['home_score0']+rnd1_scores['home_score1'])/2
rnd1_scores['away_score'] = (rnd1_scores['away_score0']+rnd1_scores['away_score1'])/2

In [146]:
rnd1_frcst = rnd1_teams.join(rnd1_scores)
teams = pd.read_pickle('.\\data\\MTeams.pkl')
teams['home_team'] = teams['TeamName']
teams['away_team'] = teams['TeamName']
away_teams = teams.loc[:, ['TeamID', 'away_team']].copy()
home_teams = teams.loc[:, ['TeamID', 'home_team']].copy()

In [147]:
rnd1_frcst_1 = rnd1_frcst.merge(away_teams, how='left', left_on = 'away_id', right_on='TeamID')
rnd1_frcst_2 = rnd1_frcst_1.merge(home_teams, how='left', left_on = 'home_id', right_on = 'TeamID')

In [148]:
rnd1_frcst_2['total'] = rnd1_frcst_2['home_score'] + rnd1_frcst_2['away_score']

In [149]:
rnd1_frcst_2.loc[rnd1_frcst_2['away_score'] > rnd1_frcst_2['home_score']]

,home_id,away_id,away_score,home_score,TeamID_x,away_team,TeamID_y,home_team,total
12,1281.0,1429.0,83.061424,81.971291,1429,Utah St,1281,Missouri,165.032715
20,1234.0,1120.0,80.950592,79.359253,1120,Auburn,1234,Iowa,160.309845
28,1192.0,1411.0,74.912872,73.680161,1411,TX Southern,1192,F Dickinson,148.593033
29,1369.0,1394.0,78.415154,76.805016,1394,TAM C. Christi,1369,SE Missouri St,155.220169
31,1113.0,1305.0,66.430870,66.122818,1305,Nevada,1113,Arizona St,132.553680


In [174]:
rnd1_df = pd.read_excel('..\\round1_team_id.xlsx', engine='openpyxl')
rnd1_df['season'] = 2023

round1_0 = rnd1_df.merge(kp_home, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
round1_1 = round1_0.merge(kp_away, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
round1_2 = rnd1_df.merge(kp_away, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
round1_3 = round1_2.merge(kp_home, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
rnd1_x = round1_1.loc[:, x_cols]

rnd1_x[x_cols] = rnd1_x[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

rnd1_x1 = round1_3.loc[:, x_cols]

rnd1_x1[x_cols] = rnd1_x1[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

rnd1_away = regressor_away.predict(rnd1_x)
rnd1_home = regressor.predict(rnd1_x)
rnd1_away2 = regressor_away.predict(rnd1_x1)
rnd1_home2 = regressor.predict(rnd1_x1)
rnd1_teams = round1_1[['home_id', 'away_id']]
rnd1_scores = pd.DataFrame({'away_score0' : rnd1_away,
                            'away_score1' : rnd1_away2,
                            'home_score0' : rnd1_home, 
                            'home_score1' : rnd1_home2})

rnd1_scores['home_score'] = (rnd1_scores['home_score0']+rnd1_scores['home_score1'])/2
rnd1_scores['away_score'] = (rnd1_scores['away_score0']+rnd1_scores['away_score1'])/2

rnd1_frcst = rnd1_teams.join(rnd1_scores)
teams = pd.read_pickle('.\\data\\MTeams.pkl')
teams['home_team'] = teams['TeamName']
teams['away_team'] = teams['TeamName']
away_teams = teams.loc[:, ['TeamID', 'away_team']].copy()
home_teams = teams.loc[:, ['TeamID', 'home_team']].copy()
rnd1_frcst_1 = rnd1_frcst.merge(away_teams, how='left', left_on = 'away_id', right_on='TeamID')
rnd1_frcst_2 = rnd1_frcst_1.merge(home_teams, how='left', left_on = 'home_id', right_on = 'TeamID')
rnd1_frcst_2['total'] = rnd1_frcst_2['home_score'] + rnd1_frcst_2['away_score']
rnd1_frcst_2.to_excel('..\\round1_output.xlsx', index=False, engine = 'openpyxl')

In [176]:
rnd1_df = pd.read_excel('..\\round2_team_id.xlsx', engine='openpyxl')
rnd1_df['season'] = 2023

round1_0 = rnd1_df.merge(kp_home, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
round1_1 = round1_0.merge(kp_away, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
round1_2 = rnd1_df.merge(kp_away, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
round1_3 = round1_2.merge(kp_home, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
rnd1_x = round1_1.loc[:, x_cols]

rnd1_x[x_cols] = rnd1_x[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

rnd1_x1 = round1_3.loc[:, x_cols]

rnd1_x1[x_cols] = rnd1_x1[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

rnd1_away = regressor_away.predict(rnd1_x)
rnd1_home = regressor.predict(rnd1_x)
rnd1_away2 = regressor_away.predict(rnd1_x1)
rnd1_home2 = regressor.predict(rnd1_x1)
rnd1_teams = round1_1[['home_id', 'away_id']]
rnd1_scores = pd.DataFrame({'away_score0' : rnd1_away,
                            'away_score1' : rnd1_away2,
                            'home_score0' : rnd1_home, 
                            'home_score1' : rnd1_home2})

rnd1_scores['home_score'] = (rnd1_scores['home_score0']+rnd1_scores['home_score1'])/2
rnd1_scores['away_score'] = (rnd1_scores['away_score0']+rnd1_scores['away_score1'])/2

rnd1_frcst = rnd1_teams.join(rnd1_scores)
teams = pd.read_pickle('.\\data\\MTeams.pkl')
teams['home_team'] = teams['TeamName']
teams['away_team'] = teams['TeamName']
away_teams = teams.loc[:, ['TeamID', 'away_team']].copy()
home_teams = teams.loc[:, ['TeamID', 'home_team']].copy()
rnd1_frcst_1 = rnd1_frcst.merge(away_teams, how='left', left_on = 'away_id', right_on='TeamID')
rnd1_frcst_2 = rnd1_frcst_1.merge(home_teams, how='left', left_on = 'home_id', right_on = 'TeamID')
rnd1_frcst_2['total'] = rnd1_frcst_2['home_score'] + rnd1_frcst_2['away_score']
rnd1_frcst_2.to_excel('..\\round2_output.xlsx', index=False, engine = 'openpyxl')

In [177]:
rnd1_df = pd.read_excel('..\\round3_team_id.xlsx', engine='openpyxl')
rnd1_df['season'] = 2023

round1_0 = rnd1_df.merge(kp_home, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
round1_1 = round1_0.merge(kp_away, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
round1_2 = rnd1_df.merge(kp_away, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
round1_3 = round1_2.merge(kp_home, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
rnd1_x = round1_1.loc[:, x_cols]

rnd1_x[x_cols] = rnd1_x[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

rnd1_x1 = round1_3.loc[:, x_cols]

rnd1_x1[x_cols] = rnd1_x1[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

rnd1_away = regressor_away.predict(rnd1_x)
rnd1_home = regressor.predict(rnd1_x)
rnd1_away2 = regressor_away.predict(rnd1_x1)
rnd1_home2 = regressor.predict(rnd1_x1)
rnd1_teams = round1_1[['home_id', 'away_id']]
rnd1_scores = pd.DataFrame({'away_score0' : rnd1_away,
                            'away_score1' : rnd1_away2,
                            'home_score0' : rnd1_home, 
                            'home_score1' : rnd1_home2})

rnd1_scores['home_score'] = (rnd1_scores['home_score0']+rnd1_scores['home_score1'])/2
rnd1_scores['away_score'] = (rnd1_scores['away_score0']+rnd1_scores['away_score1'])/2

rnd1_frcst = rnd1_teams.join(rnd1_scores)
teams = pd.read_pickle('.\\data\\MTeams.pkl')
teams['home_team'] = teams['TeamName']
teams['away_team'] = teams['TeamName']
away_teams = teams.loc[:, ['TeamID', 'away_team']].copy()
home_teams = teams.loc[:, ['TeamID', 'home_team']].copy()
rnd1_frcst_1 = rnd1_frcst.merge(away_teams, how='left', left_on = 'away_id', right_on='TeamID')
rnd1_frcst_2 = rnd1_frcst_1.merge(home_teams, how='left', left_on = 'home_id', right_on = 'TeamID')
rnd1_frcst_2['total'] = rnd1_frcst_2['home_score'] + rnd1_frcst_2['away_score']
rnd1_frcst_2.to_excel('..\\round3_output.xlsx', index=False, engine = 'openpyxl')

In [179]:
rnd1_df = pd.read_excel('..\\round4_team_id.xlsx', engine='openpyxl')
rnd1_df['season'] = 2023

round1_0 = rnd1_df.merge(kp_home, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
round1_1 = round1_0.merge(kp_away, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
round1_2 = rnd1_df.merge(kp_away, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
round1_3 = round1_2.merge(kp_home, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
rnd1_x = round1_1.loc[:, x_cols]

rnd1_x[x_cols] = rnd1_x[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

rnd1_x1 = round1_3.loc[:, x_cols]

rnd1_x1[x_cols] = rnd1_x1[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

rnd1_away = regressor_away.predict(rnd1_x)
rnd1_home = regressor.predict(rnd1_x)
rnd1_away2 = regressor_away.predict(rnd1_x1)
rnd1_home2 = regressor.predict(rnd1_x1)
rnd1_teams = round1_1[['home_id', 'away_id']]
rnd1_scores = pd.DataFrame({'away_score0' : rnd1_away,
                            'away_score1' : rnd1_away2,
                            'home_score0' : rnd1_home, 
                            'home_score1' : rnd1_home2})

rnd1_scores['home_score'] = (rnd1_scores['home_score0']+rnd1_scores['home_score1'])/2
rnd1_scores['away_score'] = (rnd1_scores['away_score0']+rnd1_scores['away_score1'])/2

rnd1_frcst = rnd1_teams.join(rnd1_scores)
teams = pd.read_pickle('.\\data\\MTeams.pkl')
teams['home_team'] = teams['TeamName']
teams['away_team'] = teams['TeamName']
away_teams = teams.loc[:, ['TeamID', 'away_team']].copy()
home_teams = teams.loc[:, ['TeamID', 'home_team']].copy()
rnd1_frcst_1 = rnd1_frcst.merge(away_teams, how='left', left_on = 'away_id', right_on='TeamID')
rnd1_frcst_2 = rnd1_frcst_1.merge(home_teams, how='left', left_on = 'home_id', right_on = 'TeamID')
rnd1_frcst_2['total'] = rnd1_frcst_2['home_score'] + rnd1_frcst_2['away_score']
rnd1_frcst_2.to_excel('..\\round4_output.xlsx', index=False, engine = 'openpyxl')

In [180]:
rnd1_df = pd.read_excel('..\\round5_team_id.xlsx', engine='openpyxl')
rnd1_df['season'] = 2023

round1_0 = rnd1_df.merge(kp_home, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
round1_1 = round1_0.merge(kp_away, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
round1_2 = rnd1_df.merge(kp_away, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
round1_3 = round1_2.merge(kp_home, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
rnd1_x = round1_1.loc[:, x_cols]

rnd1_x[x_cols] = rnd1_x[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

rnd1_x1 = round1_3.loc[:, x_cols]

rnd1_x1[x_cols] = rnd1_x1[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

rnd1_away = regressor_away.predict(rnd1_x)
rnd1_home = regressor.predict(rnd1_x)
rnd1_away2 = regressor_away.predict(rnd1_x1)
rnd1_home2 = regressor.predict(rnd1_x1)
rnd1_teams = round1_1[['home_id', 'away_id']]
rnd1_scores = pd.DataFrame({'away_score0' : rnd1_away,
                            'away_score1' : rnd1_away2,
                            'home_score0' : rnd1_home, 
                            'home_score1' : rnd1_home2})

rnd1_scores['home_score'] = (rnd1_scores['home_score0']+rnd1_scores['home_score1'])/2
rnd1_scores['away_score'] = (rnd1_scores['away_score0']+rnd1_scores['away_score1'])/2

rnd1_frcst = rnd1_teams.join(rnd1_scores)
teams = pd.read_pickle('.\\data\\MTeams.pkl')
teams['home_team'] = teams['TeamName']
teams['away_team'] = teams['TeamName']
away_teams = teams.loc[:, ['TeamID', 'away_team']].copy()
home_teams = teams.loc[:, ['TeamID', 'home_team']].copy()
rnd1_frcst_1 = rnd1_frcst.merge(away_teams, how='left', left_on = 'away_id', right_on='TeamID')
rnd1_frcst_2 = rnd1_frcst_1.merge(home_teams, how='left', left_on = 'home_id', right_on = 'TeamID')
rnd1_frcst_2['total'] = rnd1_frcst_2['home_score'] + rnd1_frcst_2['away_score']
rnd1_frcst_2.to_excel('..\\round5_output.xlsx', index=False, engine = 'openpyxl')

In [181]:
rnd1_df = pd.read_excel('..\\round6_team_id.xlsx', engine='openpyxl')
rnd1_df['season'] = 2023

round1_0 = rnd1_df.merge(kp_home, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
round1_1 = round1_0.merge(kp_away, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
round1_2 = rnd1_df.merge(kp_away, how='left', left_on=['season', 'home_id'], right_on = ['season', 'TeamID'])
round1_3 = round1_2.merge(kp_home, how='left', left_on=['season', 'away_id'], right_on = ['season', 'TeamID'])
rnd1_x = round1_1.loc[:, x_cols]

rnd1_x[x_cols] = rnd1_x[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

rnd1_x1 = round1_3.loc[:, x_cols]

rnd1_x1[x_cols] = rnd1_x1[x_cols].apply(pd.to_numeric, errors='coerce', axis=1)

rnd1_away = regressor_away.predict(rnd1_x)
rnd1_home = regressor.predict(rnd1_x)
rnd1_away2 = regressor_away.predict(rnd1_x1)
rnd1_home2 = regressor.predict(rnd1_x1)
rnd1_teams = round1_1[['home_id', 'away_id']]
rnd1_scores = pd.DataFrame({'away_score0' : rnd1_away,
                            'away_score1' : rnd1_away2,
                            'home_score0' : rnd1_home, 
                            'home_score1' : rnd1_home2})

rnd1_scores['home_score'] = (rnd1_scores['home_score0']+rnd1_scores['home_score1'])/2
rnd1_scores['away_score'] = (rnd1_scores['away_score0']+rnd1_scores['away_score1'])/2

rnd1_frcst = rnd1_teams.join(rnd1_scores)
teams = pd.read_pickle('.\\data\\MTeams.pkl')
teams['home_team'] = teams['TeamName']
teams['away_team'] = teams['TeamName']
away_teams = teams.loc[:, ['TeamID', 'away_team']].copy()
home_teams = teams.loc[:, ['TeamID', 'home_team']].copy()
rnd1_frcst_1 = rnd1_frcst.merge(away_teams, how='left', left_on = 'away_id', right_on='TeamID')
rnd1_frcst_2 = rnd1_frcst_1.merge(home_teams, how='left', left_on = 'home_id', right_on = 'TeamID')
rnd1_frcst_2['total'] = rnd1_frcst_2['home_score'] + rnd1_frcst_2['away_score']
rnd1_frcst_2.to_excel('..\\round6_output.xlsx', index=False, engine = 'openpyxl')